<a href="https://colab.research.google.com/github/guangyitan/Udemy-Unsupervised-Machine-Learning-with-2-Capstone-ML-Projects/blob/main/Dimensionality%20Reduction/Introduction_to_Dimensionality_Reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# mount to google drive
from google.colab import drive 
drive.mount('/content/MyDrive')
project_path = '/content/MyDrive/My Drive/Colab Notebooks/Udemy/Dimensionality Reduction/'

#read the excel file
data = pd.read_csv(project_path + "Employee_Data.csv")
data.shape

Mounted at /content/MyDrive


(1470, 35)

In [8]:
# Visualise the dataset

# set num of columns to display
pd.set_option('max_columns', 35)
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


# Removing Usless Columns

In [11]:
# look for useless columns

print("Number of Records in the Dataset: ", data.shape[0])
print("Number of Unique Values in EmployeeCount Column: ", data['EmployeeCount'].nunique())
print("Number of Unique Values in EmployeeNumber Column: ", data['EmployeeNumber'].nunique())
print("Number of Unique Values in Over18 Column: ", data['Over18'].nunique())
print("Number of Unique Values in StandardHours Column: ", data['StandardHours'].nunique())


Number of Records in the Dataset:  1470
Number of Unique Values in EmployeeCount Column:  1
Number of Unique Values in EmployeeNumber Column:  1470
Number of Unique Values in Over18 Column:  1
Number of Unique Values in StandardHours Column:  1


For EmployeeNumber Column, each row has unique values in all records.

For EmployeeCount, Over18 and StandardHours Columns, they have only one unique value in all records.

These columns are useless as they do not have any trends or patterns which can be used for predictive analytics.

In [12]:
# remove the useless columns from the dataset

data = data.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis = 1)
print("Number of Columns left: ", data.shape[1])

Number of Columns left:  31


In [14]:
# check for Missing values in the dataset
data.isnull().sum().sum()

0

There are no null/ missing values in the dataset

In [15]:
# check for columns with Object Data Type
data.select_dtypes('object').head()

,Attrition,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,OverTime
0,Yes,Travel_Rarely,Sales,Life Sciences,Female,Sales Executive,Single,Yes
1,No,Travel_Frequently,Research & Development,Life Sciences,Male,Research Scientist,Married,No
2,Yes,Travel_Rarely,Research & Development,Other,Male,Laboratory Technician,Single,Yes
3,No,Travel_Frequently,Research & Development,Life Sciences,Female,Research Scientist,Married,Yes
4,No,Travel_Rarely,Research & Development,Medical,Male,Laboratory Technician,Married,No


# Encode Object Type Data into Numerical Type Data

In [17]:
# Encode the Object Type data into Numerical Data Type
# Attrition Column is excluded as it is our Target Column
# For Attrition Column, if 'Yes', employee will leave the organization in near future
# For Attrition Column, if 'NO', employee will not leave the organization in near future

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['BusinessTravel'] = le.fit_transform(data['BusinessTravel'])
data['Department'] = le.fit_transform(data['Department'])
data['EducationField'] = le.fit_transform(data['EducationField'])
data['Gender'] = le.fit_transform(data['Gender'])
data['JobRole'] = le.fit_transform(data['JobRole'])
data['MaritalStatus'] = le.fit_transform(data['MaritalStatus'])
data['OverTime'] = le.fit_transform(data['OverTime'])

# double check to ensure all object data type are converted
data.select_dtypes('object').columns

Index(['Attrition'], dtype='object')

In [80]:
# Split dataset into x and y

# axis = 0 means along row
# axis = 1 means along column
x = data.drop(['Attrition'], axis = 1)
y = pd.DataFrame(data['Attrition'], columns = ['Attrition'])

print("shape of x", x.shape)
print("shape of y", y.shape)

shape of x (1470, 30)
shape of y (1470, 1)


# Balancing the dataset using 'SMOTEENN' sampling technique

In [81]:
# check if the dataset is balance or not 
# check on the target column which is 'Attrition' Column

data['Attrition'].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

From above we can see that the dataset has very large imbalance for **No** and **Yes** records

In [83]:
# balance the dataset distribution using 'SMOTEENN' Sampling Technique

from imblearn.combine import SMOTEENN

# defining the model
sn = SMOTEENN(random_state = 0)

# train the model
sn.fit(x, y)

# Making the samples
x, y = sn.fit_sample(x, y)

#convert y from ndarray to dataframe
y = pd.DataFrame(y, columns = ['Attrition'])

# Distribution of the dataset
y.value_counts()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, 

Attrition
Yes          814
No           774
dtype: int64

# Splitting the Dataset

In [84]:
# splitting the dataset

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

print("Shape x_train: ", x_train.shape)
print("Shape x_test: ", x_test.shape)
print("Shape y_train: ", y_train.shape)
print("Shape y_test: ", y_test.shape)

Shape x_train:  (1270, 30)
Shape x_test:  (318, 30)
Shape y_train:  (1270, 1)
Shape y_test:  (318, 1)


In [88]:
# Train a Decission Tree Classification Model

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

cr = classification_report(y_pred, y_test)
print(cr)

              precision    recall  f1-score   support

          No       0.88      0.89      0.89       157
         Yes       0.89      0.88      0.89       161

    accuracy                           0.89       318
   macro avg       0.89      0.89      0.89       318
weighted avg       0.89      0.89      0.89       318



Although the model has high recall and precision, we have to be aware that overfitting may had occured as there are many columns / features.

Thus, we have to perform dimensionality reduction to **avoid overfitting and reduce time & space complexity of our model.**